### LangChain and Neo4j

Source: </br>
https://medium.com/microsoftazure/introducing-graphrag-with-langchain-and-neo4j-90446df17c1e  </br>
https://archive.is/1scBi#selection-939.0-1025.20

### Code

In [13]:
import warnings
warnings.filterwarnings("ignore")


In [ ]:
from neo4j import GraphDatabase
from pyvis.network import Network
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain.chains import GraphCypherQAChain, RetrievalQA
from langchain_experimental.graph_transformers import LLMGraphTransformer
from yfiles_jupyter_graphs import GraphWidget


from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()


# Access the variables
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

open_api_key = os.getenv("OPEN_API_KEY")


In [2]:

# Initialize the LLM
llm = ChatOpenAI(
    model="gpt-4",
    api_key=open_api_key
)

# Initialize the LLMTransformer model
llm_transformer = LLMGraphTransformer(llm=llm)

# Transforming documents
text = """
Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly twice the usual amount of neck, which came in very useful as she
spent so much of her time craning over garden fences, spying on the
neighbors. The Dursleys had a small son called Dudley and in their
opinion there was no finer boy anywhere.
The Dursleys had everything they wanted, but they also had a secret, and
their greatest fear was that somebody would discover it. They didn't
think they could bear it if anyone found out about the Potters. Mrs.
Potter was Mrs. Dursley's sister, but they hadn't met for several years;
in fact, Mrs. Dursley pretended she didn't have a sister, because her
sister and her good-for-nothing husband were as unDursleyish as it was
possible to be. The Dursleys shuddered to think what the neighbors would
say if the Potters arrived in the street. The Dursleys knew that the
Potters had a small son, too, but they had never even seen him. This boy
was another good reason for keeping the Potters away; they didn't want
Dudley mixing with a child like that.
"""
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")


Nodes:[Node(id='Mrs. Potter', type='Person'), Node(id='number four, Privet Drive', type='Location'), Node(id='Dudley', type='Person'), Node(id='Mr. Dursley', type='Person'), Node(id='The Potters', type='Family'), Node(id='Mrs. Dursley', type='Person'), Node(id='Unnamed Boy', type='Person'), Node(id='The Dursleys', type='Family'), Node(id='unknown', type='Unknown'), Node(id='drills', type='Product'), Node(id='Grunnings', type='Company')]
Relationships:[Relationship(source=Node(id='Mr. Dursley', type='Person'), target=Node(id='Mrs. Dursley', type='Person'), type='IS_MARRIED_TO'), Relationship(source=Node(id='Mr. Dursley', type='Person'), target=Node(id='number four, Privet Drive', type='Location'), type='LIVES_AT'), Relationship(source=Node(id='Mrs. Dursley', type='Person'), target=Node(id='number four, Privet Drive', type='Location'), type='LIVES_AT'), Relationship(source=Node(id='Mr. Dursley', type='Person'), target=Node(id='Grunnings', type='Company'), type='WORKS_FOR'), Relationship(

In [3]:
# Add graph documents to Neo4j
graph = Neo4jGraph()
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

###  graphical representation of our DB

In [4]:
# Show the graph
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    driver = GraphDatabase.driver(
        uri=os.environ["NEO4J_URI"],
        auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"])
    )
    session = driver.session()
    widget = GraphWidget(graph=session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    return widget

showGraph()

GraphWidget(layout=Layout(height='700px', width='100%'))

### vector search feature

In [5]:
# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    api_key=open_api_key
)

In [6]:
# Create vector index
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

### vector_index query

In [7]:
# Perform similarity search
query = "Who is Dudley?"
results = vector_index.similarity_search(query, k=1)
print(results[0].page_content)


text: 
Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly twice the usual amount of neck, which came in very useful as she
spent so much of her time craning over garden fences, spying on the
neighbors. The Dursleys had a small son called Dudley and in their
opinion there was no finer boy anywhere.
The Dursleys had everything they wanted, but they also had a secret, and
their greatest fear was that somebody would discover it. They didn't
think they could bear it if anyone found out about the Potters. Mrs.
Potter was Mrs. Dursley's sister, but they hadn't met for sever

### Neo4j’s Cypher query language

In [8]:
# Graph Cypher QA Chain
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)
response = chain.invoke({"query": "What is Mr. Dursley's job?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {id: "Mr. Dursley"})-[:WORKS_FOR]->(c:Company) RETURN c.id
Full Context:
[{'c.id': 'Grunnings'}]

> Finished chain.


{'query': "What is Mr. Dursley's job?",
 'result': 'Mr. Dursley works at Grunnings.'}

### Leveraging a classic QA chain and using the method vector_index.as_retriever() that can be applied to data stores in LangChain (both vectordb and graphdb).

In [ ]:
# RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_index.as_retriever(),
    verbose=True
)

# Use invoke instead of call
result = qa_chain.invoke({"query": "What is Mr. Dursley's job?"})
result["result"]



> Entering new RetrievalQA chain...

> Finished chain.


'Mr. Dursley is the director of a firm called Grunnings, which makes drills.'